Scenario: Multiple data scientists working on multiple ML models

MLflow setup:

Tracking Server: Remote(EC2)
Backend Store: RDS
Aritfact Store: S3

In [1]:
import mlflow

In [2]:
import os
os.environ["AWS_PROFILE"] = "mlflow"

In [3]:
import boto3
session = boto3.session.Session()
print("Using AWS profile:", session.profile_name)

s3 = session.client("s3")
print([b["Name"] for b in s3.list_buckets()["Buckets"]])

Using AWS profile: mlflow
['mlflow-artifacts-1988']


In [4]:
mlflow.set_tracking_uri("http://34.203.234.36:5000/")

In [5]:
mlflow.get_tracking_uri()

'http://34.203.234.36:5000/'

In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-1988/1', creation_time=1752992979572, experiment_id='1', last_update_time=1752992979572, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-1988/0', creation_time=1752992523064, experiment_id='0', last_update_time=1752992523064, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
os.environ["AWS_PROFILE"] = "mlflow"

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/07/20 07:00:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/20 07:00:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 's3://mlflow-artifacts-1988/1/4e899ac410aa4d7a8c9d6e714346420e/artifacts'
🏃 View run casual-wasp-286 at: http://34.203.234.36:5000/#/experiments/1/runs/4e899ac410aa4d7a8c9d6e714346420e
🧪 View experiment at: http://34.203.234.36:5000/#/experiments/1


Interact with mlflow to register the model

In [8]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://34.203.234.36:5000/")

In [9]:
client.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-1988/1', creation_time=1752992979572, experiment_id='1', last_update_time=1752992979572, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-1988/0', creation_time=1752992523064, experiment_id='0', last_update_time=1752992523064, lifecycle_stage='active', name='Default', tags={}>]

In [10]:
run_id = client.search_runs(experiment_ids=['1'])[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/07/20 07:04:57 WARNING mlflow.tracking._model_registry.fluent: Run with id 4e899ac410aa4d7a8c9d6e714346420e has no artifacts at artifact path 'models', registering model based on models:/m-eb2198ef72dd40df9686610593bf179e instead
2025/07/20 07:04:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1752995097712, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1752995097712, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='4e899ac410aa4d7a8c9d6e714346420e', run_link='', source='models:/m-eb2198ef72dd40df9686610593bf179e', status='READY', status_message=None, tags={}, user_id='', version='1'>